### Jacobian_confirmation-Optimized  

Objective is to test the Jacobian and approximate Hessian. Testing Analytic 3x3 and comupted $\mathbf{J}^H\mathbf{J}$ from analytic $\mathbf{J}$.

In [1]:
import numpy as np 
import ProjectPacks as pp
import matplotlib.pyplot as plt 
%matplotlib inline

**Confirmation of JHJ**

In [4]:
Amp,l,m,ut,vt,arrayHxpos = pp.formatSParams('Array_Profile.txt', 'Field_Profile.txt', td= 100)
ut, vt= ut[:3,:3], vt[:3,:3]


SNum= len(Amp)
ArNum= len(ut) ## ut will always be a square mat

In [5]:
print(np.shape(ut))

(3, 3, 100)


In [6]:
# Analytic JHJ

Complete= np.zeros((len(ut),len(ut)), dtype=complex)
Complete_An= np.zeros((len(ut),len(ut)), dtype=complex)
for i in [21]: #in range(100):

    U= np.array(ut)[:,:,i]
    V= np.array(vt)[:,:,i]

    sumU,sqsumU= np.sum(np.triu(U,1)),np.sum(np.square(np.triu(U,1)))
    sumV, sqsumV= np.sum(np.triu(V,1)),np.sum(np.square(np.triu(V,1)))
    sumUV= np.sum(np.multiply(np.triu(U,1),np.triu(V,1)))

    ## Packing into Analytic JHJ
    JHJ= np.array([[3, -2j*np.pi*Amp[0]*sumU, -2j*np.pi*Amp[0]*sumV],
                  [2j*np.pi*Amp[0]*sumU, np.square(2*np.pi*Amp[0])*sqsumU, np.square(2*np.pi*Amp[0])*sumUV],
                  [2j*np.pi*Amp[0]*sumV, np.square(2*np.pi*Amp[0])*sumUV, np.square(2*np.pi*Amp[0])*sqsumV]])

    ## Calculating JHJ explicitly from J
    Psi= 2j*np.pi*(U*l[0]+V*m[0])
    expPsi= lambda unicoeff: np.exp(unicoeff*Psi)
    coeU= -2j*np.pi*Amp[0]*U
    coeV= -2j*np.pi*Amp[0]*V

    ## remember the -(minus) for negative exponents

    Jkt= np.array([expPsi(-1)[np.nonzero(np.triu(U,1))],
                   (coeU*expPsi(-1))[np.nonzero(np.triu(U,1))],(coeV*expPsi(-1))[np.nonzero(np.triu(V,1))]])
    

    ## NNNNBBBB!!!! with the above we need to be careful because indexing goes from left to right not up to down
    ## in the 3x3 case it doesnt matter but in the long run we need to make it more general

    J= Jkt.T # since there Jkt=Jktbar we don't need to concatenate anything
    Jh= J.conj().T ## because .H and .getH only works for matrices not ndarrays
    JhJ= np.dot(Jh,J) ## this is not element-wise, this is equivalent of matrix multiplication

    
    Complete += JhJ
    Complete_An += JHJ
    
   # print(np.allclose(JHJ, JhJ, rtol=1e-15, atol=1e-15, equal_nan=False))

In [7]:
print("\n Analytic JHJ: \n",Complete_An)
print("\n Computed JHJ: \n",Complete)
print("\n Analytic JHJinv: \n",np.linalg.pinv(Complete))
print('\n Analytic JHJ p(suedo) inv \n',np.linalg.pinv(JHJ).dot(JHJ))
print("\n Analytic JHJ = computed JHJ? \n",np.allclose(JHJ, JhJ, rtol=1e-15, atol=1e-15, equal_nan=False))


 Analytic JHJ: 
 [[  3.00000000e+00   +0.j           0.00000000e+00-4350.08835489j
    0.00000000e+00+5013.32066842j]
 [  0.00000000e+00+4350.08835489j   7.24406380e+06   +0.j
   -8.34852348e+06   +0.j        ]
 [  0.00000000e+00-5013.32066842j  -8.34852348e+06   +0.j
    9.62137361e+06   +0.j        ]]

 Computed JHJ: 
 [[  3.00000000e+00   +0.j           0.00000000e+00-4350.08835489j
    0.00000000e+00+5013.32066842j]
 [  0.00000000e+00+4350.08835489j   7.24406380e+06   +0.j
   -8.34852348e+06   +0.j        ]
 [  0.00000000e+00-5013.32066842j  -8.34852348e+06   +0.j
    9.62137361e+06   +0.j        ]]

 Analytic JHJinv: 
 [[  2.57894737e+00 -1.06672743e-16j  -6.98794539e-17 +6.65185764e-04j
   -6.06986540e-17 -7.66602720e-04j]
 [  4.06867307e-14 -6.65187858e-04j   1.97038898e-07 +1.05123262e-17j
   -2.27080258e-07 -1.20786506e-17j]
 [  3.53042106e-14 +7.66600903e-04j  -2.27079167e-07 +9.08520599e-18j
    2.61700590e-07 -1.05123556e-17j]]

 Analytic JHJ p(suedo) inv 
 [[  1.00000000e

**Report**  
To a tolerance of 15 decimal places, JHJ is computed correctly, I am satisfied with this error. 